# Лабораторная работа 6, Самсонов Савелий Артёмович М8О-406Б-21

### Выбор датасета

Kuzushiji-MNIST (KMNIST) — это набор данных, содержащий изображения японских курсивных символов из классической литературы, представленные в том же формате, что и известный MNIST (28x28 пикселей, оттенки серого, 10 классов). Вот ключевые направления, где он может быть полезен на практике:

1. Распознавание исторических рукописей (OCR для архивных документов)

    Многие японские исторические тексты написаны курсивом (Kuzushiji), который сложен для автоматического распознавания. Современные OCR (Tesseract, ABBYY) плохо работают с такими символами.

    Возможное применение KMNIST:
    - Оцифровка архивов библиотек (Национальная Diet Library Японии) или музеев.

2. Поддержка исследований в Digital Humanities

    Лингвисты и историки анализируют рукописи вручную, что требует огромного времени.
    
    Возможное применение KMNIST:
    - Автоматическая категоризация символов в больших корпусах текстов (например, поиск цитат из классической литературы).
    - Визуальный поиск по стилю написания (аналогично тому, как работают поисковики по искусству).

3. Тестирование robustness моделей CV

    Современные классификаторы часто переобучаются на "идеальных" данных (типа MNIST), но падают в реальных условиях (размытые, наклонные символы).
    
    Возможное применение KMNIST:
    - Валидация моделей для сценариев с "шумными" данными (например, распознавание рукописных медицинских форм или подписей).
    - Сравнение методов аугментации (эластичные деформации, добавление фона).

### Выбор метрик для классификации

1. Accuracy, измеряет долю правильно классифицированных экземпляров от общего числа примеров. Простая и понятная метрика, но может давать ошибочное представление для несбалансированных классов (когда классы имеют существенно отличающееся количество экземпляров).
2. Precision, измеряет долю правильных положительных предсказаний среди всех предсказанных положительных примеров.
3. Recall, измеряет долю правильно предсказанных положительных примеров среди всех реальных положительных примеров.
Precision, Recall важны в случае несбалансированных классов и при необходимости минимизировать ложные срабатывания.
4. F1-мера, является гармоническим средним между точностью и полнотой и используется для сбалансирования этих двух метрик. Комбинирует точность и полноту, идеально подходит для несбалансированных задач.

## 2. Создание бейзлайна и оценка качества

Импорт библиотек

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

### Обучение сверточной модели и оценка качества по выбранным метрикам

Подготовка данных

In [ ]:
transform = transforms.Compose([
    transforms.Resize(224),  # Изменяем размер изображений до 224x224, так как ResNet ожидает такие размеры
    transforms.Grayscale(num_output_channels=3),  # Конвертируем изображение в 3 канала (RGB), т.к. ResNet работает с 3 каналами
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

trainset = torchvision.datasets.KMNIST(root='./data2', train=True, download=True, transform=transform)
testset = torchvision.datasets.KMNIST(root='./data2', train=False, download=True, transform=transform)

subset_size = 30000
train_subset = Subset(trainset, range(subset_size))

trainloader = DataLoader(train_subset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

Определяем модель

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Обучение модели

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(trainloader):.4f}, Точность: {100 * correct / total:.2f}%")


Эпоха [1/5], Потери: 0.1373, Точность: 95.74%
Эпоха [2/5], Потери: 0.0432, Точность: 98.65%
Эпоха [3/5], Потери: 0.0353, Точность: 98.89%
Эпоха [4/5], Потери: 0.0191, Точность: 99.39%
Эпоха [5/5], Потери: 0.0216, Точность: 99.29%


Оценка модели

In [ ]:
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Accuracy: 0.9691
Precision: 0.9701
Recall: 0.9691
F1-score: 0.9691


### Обучение трансформерной модели и оценка качества по выбранным метрикам

Подготовка данных

In [ ]:
transform_trans = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # преобразуем в 3 канала, как для RGB изображений
    transforms.Resize((224, 224)),  # размер изображения для ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

trainset_trans = torchvision.datasets.KMNIST(root='./data3', train=True, download=True, transform=transform_trans)
testset_trans = torchvision.datasets.KMNIST(root='./data3', train=False, download=True, transform=transform_trans)

subset_size = 5000
train_subset_trans = Subset(trainset_trans, range(subset_size))
subset_size = 2000
test_subset_trans = Subset(testset_trans, range(subset_size))

trainloader_trans = DataLoader(train_subset_trans, batch_size=16, shuffle=True)
testloader_trans = DataLoader(test_subset_trans, batch_size=16, shuffle=False)

Определяем модель

In [ ]:
torch.cuda.empty_cache()

model_trans = models.vit_b_16(pretrained=True)
model_trans.heads.head = nn.Linear(model_trans.heads.head.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_trans = model_trans.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_trans.parameters(), lr=0.001)

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Обучение модели

In [ ]:
torch.cuda.empty_cache()

num_epochs = 5

for epoch in range(num_epochs):
    model_trans.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_trans(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    torch.cuda.empty_cache()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(trainloader_trans):.4f}, Точность: {100 * correct / total:.2f}%")

Эпоха [1/5], Потери: 2.2242, Точность: 16.30%
Эпоха [2/5], Потери: 2.1328, Точность: 20.28%
Эпоха [3/5], Потери: 1.9567, Точность: 27.10%
Эпоха [4/5], Потери: 1.7911, Точность: 34.06%
Эпоха [5/5], Потери: 1.6813, Точность: 38.56%


Оценка модели

In [ ]:
model_trans.eval()

all_labels_trans = []
all_preds_trans = []

with torch.no_grad():
    for inputs, labels in testloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_trans(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels_trans.extend(labels.cpu().numpy())
        all_preds_trans.extend(predicted.cpu().numpy())

accuracy_trans = accuracy_score(all_labels_trans, all_preds_trans)
precision_trans = precision_score(all_labels_trans, all_preds_trans, average='weighted')
recall_trans = recall_score(all_labels_trans, all_preds_trans, average='weighted')
f1_trans = f1_score(all_labels_trans, all_preds_trans, average='weighted')

print(f'Accuracy: {accuracy_trans:.4f}')
print(f'Precision: {precision_trans:.4f}')
print(f'Recall: {recall_trans:.4f}')
print(f'F1-score: {f1_trans:.4f}')

Accuracy: 0.3140
Precision: 0.3752
Recall: 0.3140
F1-score: 0.3125


## 3. Улучшение бейзлайна

### Гипотезы

1. Аугментация данных

    Аугментация данных может значительно улучшить способность модели обобщать, снижая вероятность переобучения. Можно добавить следующие виды аугментации:
    - Геометрические преобразования: вращение, сдвиг, масштабирование, изменение перспективы.
    - Шум: добавление гауссовского шума или случайных искажений.
    - Зеркальные отражения или случайные обрезки.

2. Использование дополнительных слоев

    Можно добавить дополнительные слои для извлечения более сложных признаков. Например, можно добавить слой Batch Normalization перед слоями активации или добавить несколько полносвязных слоев в конце сети для улучшения классификации. Добавление слоев Dropout в модель может помочь избежать чрезмерного подгона модели под тренировочные данные.


3. Смена оптимизатора:

    В данный момент используется Adam, но можно попробовать другие оптимизаторы, например, SGD с моментумом. Adam хорошо работает в большинстве случаев, но иногда SGD может привести к лучшему результату.

### Свёрточная модель

##### Аугментация данных

In [ ]:
new_transform = transforms.Compose([
    transforms.RandomRotation(10),  # Вращение на случайный угол в пределах 10 градусов
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Случайный сдвиг по осям
    transforms.RandomHorizontalFlip(),  # Случайное горизонтальное отражение
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


Подготовка данных

In [ ]:
new_trainset = torchvision.datasets.KMNIST(root='./data5', train=True, download=True, transform=new_transform)
new_testset = torchvision.datasets.KMNIST(root='./data5', train=False, download=True, transform=new_transform)

subset_size = 30000
new_train_subset = Subset(new_trainset, range(subset_size))

new_trainloader = DataLoader(new_train_subset, batch_size=64, shuffle=True)
new_testloader = DataLoader(new_testset, batch_size=64, shuffle=False)

Определяем модель

In [ ]:
new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Linear(new_model.fc.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_model.parameters(), lr=0.001)

Обучение модели

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    new_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in new_trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = new_model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(new_trainloader):.4f}, Точность: {100 * correct / total:.2f}%")


Эпоха [1/5], Потери: 0.2579, Точность: 91.95%
Эпоха [2/5], Потери: 0.1086, Точность: 96.55%
Эпоха [3/5], Потери: 0.0820, Точность: 97.51%
Эпоха [4/5], Потери: 0.0643, Точность: 97.93%
Эпоха [5/5], Потери: 0.0560, Точность: 98.21%


Оценка модели

In [ ]:
new_model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in new_testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = new_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Accuracy: 0.9363
Precision: 0.9379
Recall: 0.9363
F1-score: 0.9363


##### Дополнительные слои

In [ ]:
new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.BatchNorm1d(new_model.fc.in_features),
    nn.Linear(new_model.fc.in_features, 10)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_model.parameters(), lr=0.001)


C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Обучение модели

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    new_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = new_model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(trainloader):.4f}, Точность: {100 * correct / total:.2f}%")


Эпоха [1/5], Потери: 0.1521, Точность: 95.60%
Эпоха [2/5], Потери: 0.0551, Точность: 98.30%
Эпоха [3/5], Потери: 0.0298, Точность: 99.13%
Эпоха [4/5], Потери: 0.0228, Точность: 99.28%
Эпоха [5/5], Потери: 0.0268, Точность: 99.15%


Оценка модели

In [ ]:
new_model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = new_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Accuracy: 0.9618
Precision: 0.9624
Recall: 0.9618
F1-score: 0.9617


##### Оптимизатор

In [ ]:
new_model = models.resnet18(pretrained=True)
new_model.fc = nn.Linear(new_model.fc.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(new_model.parameters(), lr=0.001, momentum=0.9)


Обучение модели

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    new_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = new_model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(trainloader):.4f}, Точность: {100 * correct / total:.2f}%")


Эпоха [1/5], Потери: 0.4565, Точность: 87.92%
Эпоха [2/5], Потери: 0.0888, Точность: 97.84%
Эпоха [3/5], Потери: 0.0461, Точность: 99.02%
Эпоха [4/5], Потери: 0.0275, Точность: 99.48%
Эпоха [5/5], Потери: 0.0178, Точность: 99.76%


Оценка модели

In [ ]:
new_model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = new_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Accuracy: 0.9574
Precision: 0.9580
Recall: 0.9574
F1-score: 0.9573


Проверка гипотез не принесла улучшений

### Трансформерная модель

##### Аугментация данных

In [ ]:
new_transform_trans = transforms.Compose([
    transforms.RandomRotation(10),  # Вращение на случайный угол в пределах 10 градусов
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Случайный сдвиг по осям
    transforms.RandomHorizontalFlip(),  # Случайное горизонтальное отражение

    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

new_trainset_trans = torchvision.datasets.KMNIST(root='./data8', train=True, download=True, transform=new_transform_trans)
new_testset_trans = torchvision.datasets.KMNIST(root='./data8', train=False, download=True, transform=new_transform_trans)

subset_size = 5000
new_train_subset_trans = Subset(new_trainset_trans, range(subset_size))
subset_size = 2000
new_test_subset_trans = Subset(new_testset_trans, range(subset_size))

new_trainloader_trans = DataLoader(new_train_subset_trans, batch_size=16, shuffle=True)
new_testloader_trans = DataLoader(new_test_subset_trans, batch_size=16, shuffle=False)

100%|██████████| 18.2M/18.2M [00:16<00:00, 1.10MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 138kB/s]
100%|██████████| 3.04M/3.04M [00:04<00:00, 683kB/s] 
100%|██████████| 5.12k/5.12k [00:00<00:00, 9.71MB/s]


Определим модель

In [ ]:
torch.cuda.empty_cache()

model_trans = models.vit_b_16(pretrained=True)
model_trans.heads.head = nn.Linear(model_trans.heads.head.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_trans = model_trans.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_trans.parameters(), lr=0.001)

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Обучение модели

In [ ]:
torch.cuda.empty_cache()

num_epochs = 5

for epoch in range(num_epochs):
    model_trans.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in new_trainloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_trans(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    torch.cuda.empty_cache()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(new_trainloader_trans):.4f}, Точность: {100 * correct / total:.2f}%")

Эпоха [1/5], Потери: 2.4101, Точность: 10.30%
Эпоха [2/5], Потери: 2.3509, Точность: 10.46%
Эпоха [3/5], Потери: 2.3346, Точность: 11.00%
Эпоха [4/5], Потери: 2.3116, Точность: 10.98%
Эпоха [5/5], Потери: 2.2888, Точность: 12.36%


Оценка модели

In [ ]:
model_trans.eval()

all_labels_trans = []
all_preds_trans = []

with torch.no_grad():
    for inputs, labels in new_testloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_trans(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels_trans.extend(labels.cpu().numpy())
        all_preds_trans.extend(predicted.cpu().numpy())

accuracy_trans = accuracy_score(all_labels_trans, all_preds_trans)
precision_trans = precision_score(all_labels_trans, all_preds_trans, average='weighted')
recall_trans = recall_score(all_labels_trans, all_preds_trans, average='weighted')
f1_trans = f1_score(all_labels_trans, all_preds_trans, average='weighted')

print(f'Accuracy: {accuracy_trans:.4f}')
print(f'Precision: {precision_trans:.4f}')
print(f'Recall: {recall_trans:.4f}')
print(f'F1-score: {f1_trans:.4f}')

Accuracy: 0.1560
Precision: 0.0841
Recall: 0.1560
F1-score: 0.0968


C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Оптимизатор

In [ ]:
torch.cuda.empty_cache()

model_trans = models.vit_b_16(pretrained=True)

model_trans.heads.head = nn.Linear(model_trans.heads.head.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_trans = model_trans.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model_trans.parameters(), lr=0.001, weight_decay=0.01)

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Обучение модели

In [ ]:
torch.cuda.empty_cache()

num_epochs = 5

for epoch in range(num_epochs):
    model_trans.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in trainloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model_trans(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    torch.cuda.empty_cache()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(trainloader_trans):.4f}, Точность: {100 * correct / total:.2f}%")

Эпоха [1/5], Потери: 2.3534, Точность: 12.12%
Эпоха [2/5], Потери: 2.2498, Точность: 14.62%
Эпоха [3/5], Потери: 2.1256, Точность: 19.84%
Эпоха [4/5], Потери: 1.9146, Точность: 29.38%
Эпоха [5/5], Потери: 1.8086, Точность: 34.06%


Оценка модели

In [ ]:
model_trans.eval()

all_labels_trans = []
all_preds_trans = []

with torch.no_grad():
    for inputs, labels in testloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_trans(inputs)
        _, predicted = torch.max(outputs.data, 1)

        all_labels_trans.extend(labels.cpu().numpy())
        all_preds_trans.extend(predicted.cpu().numpy())

accuracy_trans = accuracy_score(all_labels_trans, all_preds_trans)
precision_trans = precision_score(all_labels_trans, all_preds_trans, average='weighted')
recall_trans = recall_score(all_labels_trans, all_preds_trans, average='weighted')
f1_trans = f1_score(all_labels_trans, all_preds_trans, average='weighted')

print(f'Accuracy: {accuracy_trans:.4f}')
print(f'Precision: {precision_trans:.4f}')
print(f'Recall: {recall_trans:.4f}')
print(f'F1-score: {f1_trans:.4f}')

Accuracy: 0.2895
Precision: 0.3610
Recall: 0.2895
F1-score: 0.2746


Проверка гипотез не принесла улучшений

### Выводы

Улучшить результаты не удалось.

Сверточная модель и так очень точна, и её версии с примененными улучшениями выдают примерно те же результаты.

Трансформерная модель, по все видимости, слишком мало обучалась. Для скорости обучения было взято малое число эпох, для улучшения результатов необходимо, как минимум, увеличить его.

## 4. Имплементация алгоритма машинного обучения 

### Сверточная модель

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Подготовка данных

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

cust_trainset = torchvision.datasets.KMNIST(root='./data6', train=True, download=True, transform=transform)
cust_testset = torchvision.datasets.KMNIST(root='./data6', train=False, download=True, transform=transform)

subset_size = 30000
cust_train_subset = Subset(cust_trainset, range(subset_size))

cust_trainloader = DataLoader(cust_train_subset, batch_size=64, shuffle=True)
cust_testloader = DataLoader(cust_testset, batch_size=64, shuffle=False)

Обучение модели

In [ ]:
cust_model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cust_model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(cust_trainloader, 0):
        optimizer.zero_grad()

        outputs = cust_model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Эпоха {epoch + 1}, шаг {i + 1}, потери: {running_loss / 100:.3f}')
            running_loss = 0.0


Эпоха 1, шаг 100, потери: 0.950
Эпоха 1, шаг 200, потери: 0.366
Эпоха 1, шаг 300, потери: 0.248
Эпоха 1, шаг 400, потери: 0.206
Эпоха 2, шаг 100, потери: 0.127
Эпоха 2, шаг 200, потери: 0.113
Эпоха 2, шаг 300, потери: 0.106
Эпоха 2, шаг 400, потери: 0.111
Эпоха 3, шаг 100, потери: 0.066
Эпоха 3, шаг 200, потери: 0.068
Эпоха 3, шаг 300, потери: 0.070
Эпоха 3, шаг 400, потери: 0.062
Эпоха 4, шаг 100, потери: 0.037
Эпоха 4, шаг 200, потери: 0.037
Эпоха 4, шаг 300, потери: 0.039
Эпоха 4, шаг 400, потери: 0.043
Эпоха 5, шаг 100, потери: 0.022
Эпоха 5, шаг 200, потери: 0.024
Эпоха 5, шаг 300, потери: 0.024
Эпоха 5, шаг 400, потери: 0.027


Оценка модели

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in cust_testloader:
        outputs = cust_model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.93
Точность (Precision): 0.93
Полнота (Recall): 0.93
F1-Score: 0.93


### Трансформерная модель

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, num_classes=10, embed_dim=768, num_heads=8, num_layers=6, hidden_dim=2048):
        super(TransformerClassifier, self).__init__()

        self.conv1 = nn.Conv2d(3, embed_dim, kernel_size=16, stride=16)
        self.pos_embedding = nn.Parameter(torch.randn(1, 14*14 + 1, embed_dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

        self.transformer = nn.Transformer(
            d_model=embed_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dim_feedforward=hidden_dim
        )

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = x.flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(x.size(0), -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding

        x = self.transformer(x, x)

        x = x[:, 0]
        x = self.fc(x)

        return x


Подготовка данных

In [ ]:
cust_transform_trans = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

cust_trainset_trans = torchvision.datasets.KMNIST(root='./data3', train=True, download=True, transform=cust_transform_trans)
cust_testset_trans = torchvision.datasets.KMNIST(root='./data3', train=False, download=True, transform=cust_transform_trans)

subset_size = 5000
cust_train_subset_trans = Subset(cust_trainset_trans, range(subset_size))
subset_size = 2000
cust_test_subset_trans = Subset(cust_testset_trans, range(subset_size))

cust_trainloader_trans = DataLoader(cust_train_subset_trans, batch_size=16, shuffle=True)
cust_testloader_trans = DataLoader(cust_test_subset_trans, batch_size=16, shuffle=False)

Обучение модели

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cust_model_trans = TransformerClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cust_model_trans.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    cust_model_trans.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in cust_trainloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = cust_model_trans(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(cust_trainloader_trans):.4f}, Точность: {100 * correct / total:.2f}%")


Эпоха [1/5], Потери: 2.5110, Точность: 10.36%
Эпоха [2/5], Потери: 2.3785, Точность: 9.38%
Эпоха [3/5], Потери: 2.3394, Точность: 9.38%
Эпоха [4/5], Потери: 2.3240, Точность: 9.86%
Эпоха [5/5], Потери: 2.3200, Точность: 10.48%


Оценка модели

In [ ]:
y_true = []
y_pred = []

cust_model_trans.eval()
with torch.no_grad():
    for inputs, labels in cust_testloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = cust_model_trans(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.11
Точность (Precision): 0.01
Полнота (Recall): 0.11
F1-Score: 0.02


C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Выводы

Результаты имплементированной сверточной модели близки к результатам встроенной модели.

Как и в случае встроенных версий, трансормерная имплементированная модель получилась менее эффективной, чем сверточная, но с большей разницей, чем у встроенных версий. В целом, иного и не ожидалось в виду простоты имплементированной версии.

### Улучшения для имплементированных моделей

#### Сверточная модель

##### Аугментация данных

In [29]:
new_transform = transforms.Compose([
    transforms.RandomRotation(15),  # Поворот изображения на случайный угол
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Сдвиг изображения
    transforms.RandomHorizontalFlip(),  # Отражение по горизонтали
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


Подготовка данных

In [ ]:
new_cust_trainset = torchvision.datasets.KMNIST(root='./data7', train=True, download=True, transform=new_transform)
new_cust_testset = torchvision.datasets.KMNIST(root='./data7', train=False, download=True, transform=new_transform)

subset_size = 30000
new_cust_train_subset = Subset(new_cust_trainset, range(subset_size))

new_cust_trainloader = DataLoader(new_cust_train_subset, batch_size=64, shuffle=True)
new_cust_testloader = DataLoader(new_cust_testset, batch_size=64, shuffle=False)

100%|██████████| 18.2M/18.2M [00:10<00:00, 1.70MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 111kB/s]
100%|██████████| 3.04M/3.04M [00:03<00:00, 810kB/s] 
100%|██████████| 5.12k/5.12k [00:00<00:00, 5.15MB/s]


Обучение модели

In [ ]:
new_cust_model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_cust_model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(new_cust_trainloader, 0):
        optimizer.zero_grad()

        outputs = new_cust_model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Эпоха {epoch + 1}, шаг {i + 1}, потери: {running_loss / 100:.3f}')
            running_loss = 0.0


Эпоха 1, шаг 100, потери: 1.547
Эпоха 1, шаг 200, потери: 0.985
Эпоха 1, шаг 300, потери: 0.796
Эпоха 1, шаг 400, потери: 0.738
Эпоха 2, шаг 100, потери: 0.572
Эпоха 2, шаг 200, потери: 0.501
Эпоха 2, шаг 300, потери: 0.467
Эпоха 2, шаг 400, потери: 0.456
Эпоха 3, шаг 100, потери: 0.388
Эпоха 3, шаг 200, потери: 0.382
Эпоха 3, шаг 300, потери: 0.361
Эпоха 3, шаг 400, потери: 0.351
Эпоха 4, шаг 100, потери: 0.324
Эпоха 4, шаг 200, потери: 0.307
Эпоха 4, шаг 300, потери: 0.301
Эпоха 4, шаг 400, потери: 0.288
Эпоха 5, шаг 100, потери: 0.266
Эпоха 5, шаг 200, потери: 0.264
Эпоха 5, шаг 300, потери: 0.263
Эпоха 5, шаг 400, потери: 0.253


Оценка модели

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in new_cust_testloader:
        outputs = new_cust_model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.83
Точность (Precision): 0.83
Полнота (Recall): 0.83
F1-Score: 0.83


##### Добавление слоев

In [40]:
class ImprovedCNN(nn.Module):
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


Обучение модели

In [ ]:
new_cust_model = ImprovedCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_cust_model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(cust_trainloader, 0):
        optimizer.zero_grad()

        outputs = new_cust_model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Эпоха {epoch + 1}, шаг {i + 1}, потери: {running_loss / 100:.3f}')
            running_loss = 0.0


Эпоха 1, шаг 100, потери: 1.113
Эпоха 1, шаг 200, потери: 0.563
Эпоха 1, шаг 300, потери: 0.445
Эпоха 1, шаг 400, потери: 0.393
Эпоха 2, шаг 100, потери: 0.329
Эпоха 2, шаг 200, потери: 0.315
Эпоха 2, шаг 300, потери: 0.276
Эпоха 2, шаг 400, потери: 0.293
Эпоха 3, шаг 100, потери: 0.238
Эпоха 3, шаг 200, потери: 0.260
Эпоха 3, шаг 300, потери: 0.239
Эпоха 3, шаг 400, потери: 0.220
Эпоха 4, шаг 100, потери: 0.213
Эпоха 4, шаг 200, потери: 0.187
Эпоха 4, шаг 300, потери: 0.197
Эпоха 4, шаг 400, потери: 0.208
Эпоха 5, шаг 100, потери: 0.167
Эпоха 5, шаг 200, потери: 0.178
Эпоха 5, шаг 300, потери: 0.169
Эпоха 5, шаг 400, потери: 0.169


Оценка модели

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in cust_testloader:
        outputs = new_cust_model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.87
Точность (Precision): 0.87
Полнота (Recall): 0.87
F1-Score: 0.87


##### Оптимизатор

In [ ]:
new_cust_model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(new_cust_model.parameters(), lr=0.001, momentum=0.9)

epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(cust_trainloader, 0):
        optimizer.zero_grad()

        outputs = new_cust_model(inputs)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Эпоха {epoch + 1}, шаг {i + 1}, потери: {running_loss / 100:.3f}')
            running_loss = 0.0


Эпоха 1, шаг 100, потери: 2.259
Эпоха 1, шаг 200, потери: 2.035
Эпоха 1, шаг 300, потери: 1.397
Эпоха 1, шаг 400, потери: 0.938
Эпоха 2, шаг 100, потери: 0.699
Эпоха 2, шаг 200, потери: 0.680
Эпоха 2, шаг 300, потери: 0.609
Эпоха 2, шаг 400, потери: 0.576
Эпоха 3, шаг 100, потери: 0.529
Эпоха 3, шаг 200, потери: 0.470
Эпоха 3, шаг 300, потери: 0.441
Эпоха 3, шаг 400, потери: 0.433
Эпоха 4, шаг 100, потери: 0.383
Эпоха 4, шаг 200, потери: 0.401
Эпоха 4, шаг 300, потери: 0.345
Эпоха 4, шаг 400, потери: 0.327
Эпоха 5, шаг 100, потери: 0.303
Эпоха 5, шаг 200, потери: 0.292
Эпоха 5, шаг 300, потери: 0.290
Эпоха 5, шаг 400, потери: 0.271


Оценка модели

In [ ]:
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in cust_testloader:
        outputs = new_cust_model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.83
Точность (Precision): 0.83
Полнота (Recall): 0.83
F1-Score: 0.83


Проверка гипотез не принесла улучшений

#### Трансформерная модель

##### Аугментация данных

In [ ]:
new_cust_transform_trans = transforms.Compose([
    transforms.RandomRotation(10),  # Вращение на случайный угол в пределах 10 градусов
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Случайный сдвиг по осям
    transforms.RandomHorizontalFlip(),  # Случайное горизонтальное отражение

    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

new_cust_trainset_trans = torchvision.datasets.KMNIST(root='./data8', train=True, download=True, transform=new_cust_transform_trans)
new_cust_testset_trans = torchvision.datasets.KMNIST(root='./data8', train=False, download=True, transform=new_cust_transform_trans)

subset_size = 5000
new_cust_train_subset_trans = Subset(new_cust_trainset_trans, range(subset_size))
subset_size = 2000
new_cust_test_subset_trans = Subset(new_cust_testset_trans, range(subset_size))

new_cust_trainloader_trans = DataLoader(new_cust_train_subset_trans, batch_size=16, shuffle=True)
new_cust_testloader_trans = DataLoader(new_cust_test_subset_trans, batch_size=16, shuffle=False)

Обучение модели

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_cust_model_trans = TransformerClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(new_cust_model_trans.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    new_cust_model_trans.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in new_cust_trainloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = new_cust_model_trans(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(new_cust_trainloader_trans):.4f}, Точность: {100 * correct / total:.2f}%")


C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Эпоха [1/5], Потери: 2.4720, Точность: 10.28%
Эпоха [2/5], Потери: 2.3785, Точность: 10.46%
Эпоха [3/5], Потери: 2.3504, Точность: 10.06%
Эпоха [4/5], Потери: 2.3319, Точность: 9.92%
Эпоха [5/5], Потери: 2.3211, Точность: 9.62%


Оценка модели

In [ ]:
y_true = []
y_pred = []

new_cust_model_trans.eval()
with torch.no_grad():
    for inputs, labels in new_cust_testloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = new_cust_model_trans(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.10
Точность (Precision): 0.01
Полнота (Recall): 0.10
F1-Score: 0.02


C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Оптимизатор

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
new_cust_model_trans = TransformerClassifier().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(new_cust_model_trans.parameters(), lr=0.001, weight_decay=0.01)

C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Обучение модели

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    new_cust_model_trans.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in cust_trainloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = new_cust_model_trans(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Эпоха [{epoch+1}/{num_epochs}], Потери: {running_loss/len(cust_trainloader_trans):.4f}, Точность: {100 * correct / total:.2f}%")


Эпоха [1/5], Потери: 2.4765, Точность: 10.22%
Эпоха [2/5], Потери: 2.3742, Точность: 9.26%
Эпоха [3/5], Потери: 2.3383, Точность: 10.62%
Эпоха [4/5], Потери: 2.3179, Точность: 10.44%
Эпоха [5/5], Потери: 2.3163, Точность: 9.38%


Оценка модели

In [ ]:
y_true = []
y_pred = []

new_cust_model_trans.eval()
with torch.no_grad():
    for inputs, labels in cust_testloader_trans:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = new_cust_model_trans(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Точность: {accuracy:.2f}')
print(f'Точность (Precision): {precision:.2f}')
print(f'Полнота (Recall): {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Точность: 0.11
Точность (Precision): 0.01
Полнота (Recall): 0.11
F1-Score: 0.02


C:\Users\Gigabyte\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Проверка гипотез не принесла улучшений

### Выводы

Аналогично ситуации со встроенными моделями, улучшений не вышло.

Сверточная модель и без того обладает высокой точностью, трансформерной модели требуется серьезное улучшение архитектуры и большее обучение.